# ArrayFire Example : BlackScholes

Now let us work on a simple example, to show you what ArrayFire can do. 

### BlackScholes

This is the powerhouse of modern financial simulations. From this model, we can estimate the theoretical price of a European style option. This model is fairly used (with some modifications and tuning) by options markets participants. Now we're going to write a simple blacksholes kernel, and accelerate it using ArrayFire.

In [9]:
using ArrayFire

Write a simple blackscholes kernel.

In [11]:
function blackscholes_serial(sptprice::AbstractArray,
                           strike::AbstractArray,
                           rate::AbstractArray,
                           volatility::AbstractArray,
                           time::AbstractArray)
    logterm = log10( sptprice ./ strike)
    powterm = .5f0 .* volatility .* volatility
    den = volatility .* sqrt(time)
    d1 = (((rate .+ powterm) .* time) .+ logterm) ./ den
    d2 = d1 .- den
    NofXd1 = cndf2(d1)
    NofXd2 = cndf2(d2)
    futureValue = strike .* exp(- rate .* time)
    c1 = futureValue .* NofXd2
    call_ = sptprice .* NofXd1 .- c1
    put  = call_ .- futureValue .+ sptprice
end

@inline function cndf2(in::AbstractArray{T})
    out = 0.5f0 .+ 0.5f0 .* erf(0.707106781f0 .* in)
    return out
end

cndf2 (generic function with 1 method)

Create all the arrays: spot price, various initial strike prices, rates and volatility, and time period. We have chosen a million iterations to simulate. 

In [12]:
T = Float64
iterations = 10^6
sptprice   = T[ 42.0 for i = 1:iterations ]
initStrike = T[ 40.0 + (i / iterations) for i = 1:iterations ]
rate       = T[ 0.5 for i = 1:iterations ]
volatility = T[ 0.2 for i = 1:iterations ]
time       = T[ 0.5 for i = 1:iterations ]

1000000-element Array{Float64,1}:
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 ⋮  
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5

Let's now convert them all to GPU arrays. Now, all these different parameters now reside on the GPU. 

In [13]:
sptpriced = AFArray(sptprice)
initStriked = AFArray(initStrike)
rated = AFArray(rate)
volatilityd = AFArray(volatility)
timed = AFArray(time)

1000000-element ArrayFire.AFArray{Float64,1}:
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 ⋮  
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5
 0.5

Let's now run the black scholes simulation, and time the execution.

In [14]:
@time begin 
    put1 = blackscholes_serial(sptprice, initStrike, rate, volatility, time)
    @show sum(put1)
end

sum(put1) = 2.095482055932037e7
  0.360503 seconds (8.63 k allocations: 214.031 MB, 27.87% gc time)


2.095482055932037e7

Now let's run the accelerated blackscholes simulation. Notice that we don't need to change any code. All we need to do is feed in different inputs. 

This is **multiple dispatch** at work.

In [15]:
@time begin 
    put2 = blackscholes_serial(sptpriced, initStriked, rated, volatilityd, timed)
    @show sum(put2)
end

sum(put2) = 2.0954820559320375e7
  0.507914 seconds (328.90 k allocations: 14.810 MB)


2.0954820559320375e7